In [ ]:
!nvidia-smi

Sun Mar 16 15:23:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
#install unsloth

!pip install unsloth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 69.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import Libraries
import torch
import unsloth
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
import pandas as pd
from google.colab import drive
import wandb
from huggingface_hub import login
from getpass import getpass

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# Import required libraries
import torch
from unsloth import FastLanguageModel
from transformers import TrainingArguments, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
from trl import SFTTrainer
import wandb
import pandas as pd

In [ ]:
Verify GPU and Unsloth Installation
print("Using device:", torch.cuda.get_device_name(0))

# Enter Hugging Face Token (Manually Input in Colab)
hugging_face_token = getpass("Enter your Hugging Face Token: ")
login(hugging_face_token)

# Enter Weights & Biases (WnB) Token
wnb_token = getpass("Enter your WnB API Key: ")
wandb.login(key=wnb_token)

Using device: Tesla T4
Enter your Hugging Face Token: ··········
Enter your WnB API Key: ··········


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: natpaphat-wong12 (natpaphat-wong12-gisma-business-school) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Initialize WnB run
run = wandb.init(
    project="LLama-on-sarcasm-detection-thesis",
    job_type="zero-shot-inference",
    anonymous="allow"
)

#Load Meta-Llama-3.1-8B

In [ ]:
# Load the Pre-Trained Model

model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
max_seq_length = 2048  # Maximum sequence length
dtype = torch.float16  # Data type for inference

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True  # Use 4-bit quantization for memory efficiency
)

==((====))==  Unsloth 2025.3.14: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

#Load mistral-7b-instruct-v0.2

In [ ]:
# Load the Pre-Trained Model
model_name = "unsloth/mistral-7b-instruct-v0.2-bnb-4bit"
max_seq_length = 2048  # Maximum sequence length
dtype = torch.float16  # Data type for inference

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True  # Use 4-bit quantization for memory efficiency
)

#Prompt Engineering Framework for Sarcasm Detection (PEF-SD)

##1. Zeroshot



In [ ]:
#  Load the Dataset
dataset = pd.read_csv("/content/testing_sarcasm.csv")

# Define a Robust and Explicit Prompt Template
def generate_prompt(headline):
    return f"""
    Classify the following headline as 'Sarcastic' or 'Not Sarcastic.'
    Respond **strictly** in the following format:

    Classification: <Sarcastic or Not Sarcastic>

    Headline: "{headline}"
    """

# Run Inference on the Dataset
results = []
for i in range(len(dataset)):
    # Access the 'headline' column using .iloc
    headline = dataset.iloc[i]['headline']
    prompt = generate_prompt(headline)

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cuda")

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=50)  # Reduced tokens since explanation is removed

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract Classification
    if "Classification:" in response:
        classification = response.split("Classification:")[-1].strip()
    else:
        # Fallback if the model doesn't follow the format
        classification = "unknown"

    # Append the result
    results.append({
        "headline": headline,
        "true_label": "Sarcastic" if dataset.iloc[i]['is_sarcastic'] else "Not Sarcastic", # Access 'is_sarcastic' using .iloc
        "predicted_label": classification,
    })

# Save Results to a DataFrame
results_df = pd.DataFrame(results)

# Save DataFrame to CSV
csv_filename = "headline_sarcasm_detection_results_SAR.csv"
results_df.to_csv(csv_filename, index=False)

# Download the CSV File
from google.colab import files
files.download(csv_filename)

##2. Zeroshot withGeneral Contextual Framing

In [ ]:

#  Load the Dataset (First 100 Rows)
dataset = pd.read_csv("/content/testing_sarcasm.csv")

#  Define a Robust and Explicit Prompt Template
def generate_prompt(headline):
    return f"""
    Classify the following headline as 'Sarcastic' or 'Not Sarcastic.'

   Before you classify,please note that.Sarcasm is a form of **verbal irony**, where someone says something but **means the opposite**, often with a mocking or humorous intent.
And Important Considerations to Avoid Misclassification are Headlines using **figurative language (metaphors, hyperbole, irony)** may not always be sarcastic.
.The presence of **quotation marks** does not necessarily indicate sarcasm; they may be quoting someone.**Unexpected juxtapositions** or contrasts do not automatically imply sarcasm.Political and controversial topics should be evaluated based on intent, not just word choice.
Pay attention to **neutral and factual headlines** that might contain strong words but lack sarcastic intent.


    Respond **strictly** in the following format:

    Classification: <Sarcastic or Not Sarcastic>

    Headline: "{headline}"
    """


#  Run Inference on the Dataset
results = []
for i in range(len(dataset)):
    # Access the 'headline' column using .iloc
    headline = dataset.iloc[i]['headline']
    prompt = generate_prompt(headline)

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cuda")

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100)  # Reduced tokens since explanation is removed

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract Classification
    if "Classification:" in response:
        classification = response.split("Classification:")[-1].strip()
    else:
        # Fallback if the model doesn't follow the format
        classification = "unknown"

    # Append the result
    results.append({
        "headline": headline,
        "true_label": "Sarcastic" if dataset.iloc[i]['is_sarcastic'] else "Not Sarcastic", # Access 'is_sarcastic' using .iloc
        "predicted_label": classification,
    })

#  Save Results to a DataFrame
results_df = pd.DataFrame(results)

#  Save DataFrame to CSV
csv_filename = "Llama_BOC.csv"
results_df.to_csv(csv_filename, index=False)

#  Download the CSV File
from google.colab import files
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Few-Shot - CoT

In [ ]:

import torch

# Load the Dataset
dataset = pd.read_csv("/content/testing_sarcasm.csv")

#  Define the Minimalistic CoT Prompt Template
def generate_prompt(headline):
    return f"""
    Classify the following headline as 'Sarcastic' or 'Not Sarcastic' using step-by-step reasoning.

    **Step-by-Step Analysis:**
    1. Read the headline: "{headline}".
    2. Check for irony, exaggeration, or contradiction.
    3. Determine if the statement means the opposite of what is expected.
    5. Otherwise, classify it as **Not Sarcastic**.
 **Examples:**
    - Headline: 'tiny dog suffocates in louis vuitton bag'
      Classification: Sarcastic
    - Headline: 'gravity' & '12 years a slave' tie at 2014 pga awardsy.'
      Classification: Not Sarcastic
    - Headline:'guests' chairs tilt, spray water at them during first-ever 4d state of the union address'
      Classification: Sarcastic
    - Headline: 'stephen colbert does the real math on donald trump's promised border wall.'
      Classification: Not Sarcastic


    **Final Answer:**
    Classification:
    """

#  Define Function to Extract Classification
def extract_classification(response, headline):
    # Default value
    classification = "unknown"

    input_prompt = generate_prompt(headline)
    trimmed_response = response.replace(input_prompt, "") # Replace the input prompt with an empty placeholder

    if "Not Sarcastic" in trimmed_response:
        classification = "Not Sarcastic"
    elif "Sarcastic" in trimmed_response:
        classification = "Sarcastic"



    return classification

# Run Inference on the Dataset
results = []
for i in range(len(dataset)):
    headline = dataset.iloc[i]['headline']
    prompt = generate_prompt(headline)

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cuda")

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=200)  # Limit tokens for concise output

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the final classification from the response
    classification = extract_classification(response, headline) # Pass the headline as well, to replace the default prompt in the model response

    # Append results
    results.append({
        "headline": headline,
        "true_label": "Sarcastic" if dataset.iloc[i]['is_sarcastic'] else "Not Sarcastic",
        "predicted_label": classification,
        "model_response": response
    })

# Save Results to a DataFrame
results_df = pd.DataFrame(results)

#  Save DataFrame to CSV
csv_filename = "LLama_COT_fs_1.csv"
results_df.to_csv(csv_filename, index=False)

# Download the CSV File
from google.colab import files
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Zero-Shot - CoT

In [ ]:

import torch

#  Load the Dataset
dataset = pd.read_csv("/content/testing_sarcasm.csv")

#  Define the Minimalistic CoT Prompt Template
def generate_prompt(headline):
    return f"""
    Classify the following headline as 'Sarcastic' or 'Not Sarcastic' using step-by-step reasoning.

    **Step-by-Step Analysis:**
    1. Read the headline: "{headline}".
    2. Check for irony, exaggeration, or contradiction.
    3. Determine if the statement means the opposite of what is expected.
    5. Otherwise, classify it as **Not Sarcastic**.


    **Final Answer:**
    Classification:
    """

# Define Function to Extract Classification
def extract_classification(response, headline):
    # Default value
    classification = "unknown"

    input_prompt = generate_prompt(headline)
    trimmed_response = response.replace(input_prompt, "") # Replace the input prompt with an empty placeholder

    if "Not Sarcastic" in trimmed_response:
        classification = "Not Sarcastic"
    elif "Sarcastic" in trimmed_response:
        classification = "Sarcastic"



    return classification

# Run Inference on the Dataset
results = []
for i in range(len(dataset)):
    headline = dataset.iloc[i]['headline']
    prompt = generate_prompt(headline)

    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True).to("cuda")

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=200)  # Limit tokens for concise output

    # Decode the output
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the final classification from the response
    classification = extract_classification(response, headline) # Pass the headline as well, to replace the default prompt in the model response

    # Append results
    results.append({
        "headline": headline,
        "true_label": "Sarcastic" if dataset.iloc[i]['is_sarcastic'] else "Not Sarcastic",
        "predicted_label": classification,
        "model_response": response
    })

#  Save Results to a DataFrame
results_df = pd.DataFrame(results)

#  Save DataFrame to CSV
csv_filename = "COT_zo.csv"
results_df.to_csv(csv_filename, index=False)

#  Download the CSV File
from google.colab import files
files.download(csv_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>